<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2020ccap/2020NST_an_alligator_dying_after_100_days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
- title: style transfer demo
- source: https://qiita.com/shun_fukuchi/items/5fed6bf9d1fcf6e7e1d4?utm_source=Qiita%E3%83%8B%E3%83%A5%E3%83%BC%E3%82%B9&utm_campaign=d9da7b2b38-Qiita_newsletter_406_04_01_2020&utm_medium=email&utm_term=0_e44feaa081-d9da7b2b38-33392037
- date: 2020-0401
---

In [0]:
!wget https://thenypost.files.wordpress.com/2019/07/florida-2nd-alligaotr-attack.jpg  # 凶暴なワニの画像
!wget https://fki.ismcdn.jp/mwimgs/5/c/600m/img_5c91efc5fb6408a098e8d622cb49176698275.jpg  # 100日後に死ぬワニの画像

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
#from tensorflow import keras
from tensorflow.compat.v1 import keras as K
#from tensorflow.compat.v1 import keras as keras
#keras.__version__
from tensorflow.compat.v1.keras.preprocessing.image import load_img, img_to_array

# ターゲット画像のpath。pathは自分が保存した場所に書き換えてください。
target_image_path = 'florida-2nd-alligaotr-attack.jpg'
#target_image_path = 'img_5c91efc5fb6408a098e8d622cb49176698275.jpg'
# スタイル画像のpath。pathは自分が保存した場所に書き換えてください。
style_reference_image_path = 'img_5c91efc5fb6408a098e8d622cb49176698275.jpg'
#style_reference_image_path = 'florida-2nd-alligaotr-attack.jpg'

# 生成画像サイズ
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [0]:
import numpy as np
from tensorflow.compat.v1.keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [0]:
#from keras import backend as K

target_image = K.backend.constant(preprocess_image(target_image_path))
style_reference_image = K.backend.constant(preprocess_image(style_reference_image_path))

# 生成画像を保持するプレースホルダー
combination_image = K.backend.placeholder((1, img_height, img_width, 3))

# ３つの画像を１つのバッチにまとめる
input_tensor = K.backend.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# ３つの画像からなるバッチを入力として使用するVGG19を構築
# このモデルには学習済みのImageNetの重みが読み込まれます
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('Model loaded.')

In [0]:
#　コンテンツの損失関数
def content_loss(base, combination):
    return K.backend.sum(K.backend.square(combination - base))

#　スタイルの損失関数
def gram_matrix(x):
    features = K.backend.batch_flatten(K.backend.permute_dimensions(x, (2, 0, 1)))
    gram = K.backend.dot(features, K.backend.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.backend.sum(K.backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

#　全変動損失関数
def total_variation_loss(x):
    a = K.backend.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.backend.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.backend.sum(K.backend.pow(a + b, 1.25))

In [0]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.backend.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features, combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

In [0]:
grads = K.backend.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.backend.function([combination_image], [loss, grads])

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [0]:
from scipy.optimize import fmin_l_bfgs_b
#from scipy.misc import imsave
import imageio
import time

result_prefix = 'style_transfer_result'
iterations = 30

# Run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss.
# This is our initial state: the target image.
# Note that `scipy.optimize.fmin_l_bfgs_b` can only process flat vectors.
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration {0:03d}'.format(i), end=" ")
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:{0}'.format(min_val), end=" ")
    # Save current generated image
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    #imsave(fname, img)
    imageio.imwrite(fname, img)
    end_time = time.time()
    print('Image saved as', fname, end=" ")
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

In [0]:
from matplotlib import pyplot as plt
%matplotlib inline

original = load_img(target_image_path, target_size=(img_height, img_width))  # オリジナル
style =  load_img(style_reference_image_path, target_size=(img_height, img_width))  # スタイル画像
transfered = img  # 変換画像

fig, axs = plt.subplots(1, 3, figsize=(24, 10))
for ax, img in zip(axs, [original, style, transfered]):
    ax.axis(False)
    ax.imshow(img)
    #ax.grid(True)

plt.show()

In [0]:
result_prefix = 'nst_result'
iterations = 30

for i in range(iterations):
    print('Start of iteration {0:02d}'.format(i), end=" ")
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:{0:08.3f}'.format(min_val), end=" ")
    # Save current generated image
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '{0:02d}.png'.format(i)
    #imsave(fname, img)
    imageio.imwrite(fname, img)
    end_time = time.time()
    print('Image saved as', fname, end=" ")
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

In [0]:
from matplotlib import pyplot as plt

# コンテンツ画像
plt.figure(figsize=(12,8))
plt.imshow(load_img(target_image_path, target_size=(img_height, img_width)))
plt.axis(False)
plt.figure()

# スタイル画像
plt.figure(figsize=(12,8))
plt.imshow(load_img(style_reference_image_path, target_size=(img_height, img_width)))
plt.axis(False)
plt.figure()

# 生成画像
plt.figure(figsize=(12,8))
plt.imshow(img)
plt.axis(False)
plt.show()